## Trabajo en clase - MapReduce
**Mario Hernández 💻**

### Instalación del entorno:

In [1]:
!pip install mrjob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.6/439.6 kB 4.6 MB/s eta 0:00:00


### Ejercicio 1: Estadísticas de Texto

Completa la clase `TextStats` implementando un programa map-reduce que, dado un texto de entrada, cuente:

- El número de carácteres
- El número de líneas
- El número de palabras

y emita como resultado tres tuplas como en el ejemplo siguiente.


_Ejemplo de ejecución_

    Ancient influences have helped spawn variant interpretations
    of the nature of history which have evolved over the centuries
    and continue to change today. The modern study of history is
    wide-ranging, and includes the study of specific regions and
    the study of certain topical or thematical elements of
    historical investigation. Often history is taught as part of
    primary and secondary education, and the academic study of
    history is a major discipline in University studies
    
_Resultado_

    "chars"	472
    "lines"	8
    "words"	73

Nota: Dado un string `s` usa la función `s.split()` para obtener obtener una lista con los conjuntos de caracteres del string que están separados por espacios en blanco o usa la búsqueda por expresiones regulares vista en clase. Usa la función `len` para obtener la longitud de un string o de una lista.

**Instrucciones para Ejecutar el Código**

1. **Guardar el Código**: Guarda el código en un archivo llamado `TextStats.py`.

2. **Ejecutar el Programa**: Abre una terminal o línea de comandos y navega hasta el directorio donde guardaste `TextStats.py`. Ejecuta el siguiente comando:
   ```sh
   python TextStats.py
   ```

3. **Verificar Resultados**: El programa imprimirá las estadísticas del texto en la terminal.

**Ejemplo de Uso**

El código incluye un ejemplo de uso con un texto de varias líneas:
```python
text = """Hola, este es un ejemplo de texto.
Tiene varias líneas y palabras."""
```

El resultado de las estadísticas del texto será:
```
Estadísticas del texto:
(('chars', 52), ('lines', 2), ('words', 9))
```

Este ejemplo demuestra cómo inicializar la clase `TextStats` con un texto y cómo llamar al método `count_stats` para obtener las estadísticas de caracteres, líneas y palabras.

In [2]:
# Escribimos el código en un archivo ejecutable
%%writefile /content/TextStats.py

from mrjob.job import MRJob # Usamos la libreria mrjob para procesar los datos de manera distribuida

class TextStats(MRJob): # Definimos la clase para "heredar" las funcionalidades de MRJob

    def mapper(self, _, line): # Usamos el método mapper que toma una clave y un valor de entrada,
                              # y se produce una o más claves y valores de salida, en este caso no se usa una clave
                              # de entrada por lo que colocamos _ y el valor es asignado a cada linea o renglón del texto
                              # Self es simplemente una referencia a la instancia actual de la clase TextStats es una convención necesaria

        # Contamos carácteres, líneas y palabras mediante 3 pares clave-valor para cada línea de entrada
        yield "caracteres", len(line) # Retornamos y usamos len para saber el tamaño o número de caracteres por cada linea del texto
        yield "lineas", 1 #  Retornamos y usamos el número 1 para indicar que se ha procesado una línea del texto
        yield "palabras", len(line.split()) #  Retornamos y usamos line.split() para dividir la línea en palabras usando espacios
                                            # como separadores y luego contamos el número de palabras en total

    def reducer(self, key, values): # Empleamos el método reducer que toma la clave y una lista de valores generados
                                    # por el mapper, y produce un único valor de salida para cada clave, es decir
                                    # agrupa todos los valores asociados a una misma clave

        # Sumar los valores para cada clave
        yield key, sum(values)

if __name__ == '__main__':  # Este condicional nos indica que este archivo debe ejercutarse como un script
    TextStats.run() # Se ejecuta el MapReduce que hemos implementado


Writing /content/TextStats.py


In [3]:
# Ejecutamos el script de TextStats.py para el texto y usamos --quiet para mostrará la salida mínima necesaria
# y suprimir el resto de los mensajes de erro y otros detalles no esenciales.
%%script python TextStats.py --quiet
Ancient influences have helped spawn variant interpretations
of the nature of history which have evolved over the centuries
and continue to change today. The modern study of history is
wide-ranging, and includes the study of specific regions and
the study of certain topical or thematical elements of
historical investigation. Often history is taught as part of
primary and secondary education, and the academic study of
history is a major discipline in University studies

"lineas"	8
"palabras"	73
"caracteres"	465


**Ejemplos de aplicaciones:**
* Análisis de contenido web para optimizar SEO.
* Análisis de bases de datos documentales o de investigación científica.
* Análisis de redes sociales, examinando posts, copys, comentarios o patrones de comunicación.
* Análisis de documentos legales, procesando textos jurídicos por ejemplo o normativas.
* Procesamiento de correos electrónicos para detectar spam o clasificar contenido.

### Ejercicio 2: Número medio de palabras por línea

Completa la clase `LineAverage` para que como resultado final emita únicamente un par _clave, valor_ con el número medio de palabras por línea en el texto de entrada.

Para el mismo texto anterior, la salida debería de ser:

    "avg"	9.125

In [4]:
# Escribimos el código en un archivo ejecutable
%%writefile LineAverage.py

from mrjob.job import MRJob
import numpy as np # Importamos NumPy para realizar operaciones matemáticas

class LineAverage(MRJob):  # Definimos la clase para "heredar" las funcionalidades de MRJob

    def mapper(self, _, line): # Usamos el método mapper que toma una clave y un valor de entrada,
                              # y se produce una o más claves y valores de salida, en este caso no se usa una clave
                              # de entrada por lo que colocamos _ y el valor es asignado a cada linea o renglón del texto
                              # Self es simplemente una referencia a la instancia actual de la clase LineAverage una convención necesaria

        # Retornamos y usamos line.split() para dividir la línea en palabras usando espacios
        # como separadores y luego contamos el número de palabras en total
        yield "palabras_por_linea", len(line.split())

    def reducer(self, key, values):# Empleamos el método reducer que toma la clave y una lista de valores generados
                                      # por el mapper, y produce un único valor de salida para cada clave, es decir
                                      # agrupa todos los valores asociados a una misma clave

        lista_valores = list(values)  # Convertimos la secuencia de valores en una lista para calcular la cantidad de elementos
        total_palabras = sum(lista_valores) # Sumamos los valores de la lista, es decir el total de palabras
        total_lineas = len(lista_valores) # Contamos el número de elementos en la lista, es decir el número total de líneas
        promedio = total_palabras / total_lineas # Calculamos el promedio de palabras por línea
        yield "promedio", promedio #  Retornamos el resultado del promedio (clave-valor)

if __name__ == '__main__':
    LineAverage.run()

Writing LineAverage.py


verifica que la salida es la esperada y prueba con nuevos valores. <font style="color: red">_Elimina la opción_ `--quiet` _para ver los mensajes de error de tu código si no produce la salida esperada_

In [5]:
# Ejecutamos el script de LineAverage.py para el texto
%%script python LineAverage.py
Ancient influences have helped spawn variant interpretations
of the nature of history which have evolved over the centuries
and continue to change today. The modern study of history is
wide-ranging, and includes the study of specific regions and
the study of certain topical or thematical elements of
historical investigation. Often history is taught as part of
primary and secondary education, and the academic study of
history is a major discipline in University studies

"promedio"	9.125


No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/LineAverage.root.20240815.013439.381909
Running step 1 of 1...
reading from STDIN
job output is in /tmp/LineAverage.root.20240815.013439.381909/output
Streaming final output from /tmp/LineAverage.root.20240815.013439.381909/output...
Removing temp directory /tmp/LineAverage.root.20240815.013439.381909...


**Ejemplos de aplicaciones:**
* Análisis de documentación técnica, como procesamiento de manuales y gestión de documentos.
* Estudio de estilos de escritura o géneros literarios donde se compare autores.
* Evaluación o regulación de contenidos generados por los usuarios, examinando reseñas o comentarios.
* Estudio de artículos científicos, analizando papers académicos en repositorios digitales.

### Ejercicio 3: Índice invertido

Completa la clase `InvertedIndex` implementando un programa map-reduce que cree un indice invertido para una lista de documentos. Es decir, para cada palabra en nuestra colección de documentos el índice invertido no da en qué documentos aparece. Estos índices se usan posteriormente para responder a queries:


_Ejemplo de ejecución_ (entrada es en formato **json**)

    ["001", "the car is nice"]
    ["002", "that car is mine"]
    ["003", "that shirt is nice"]  
    ["004", "the car is the best"]
    
_Resultado_

    "best"  ["004"]
    "car"   ["001", "002", "004"]
    "is"    ["001", "002", "003", "004"]
    "mine"  ["002"]
    "nice"  ["001", "003"]
    "shirt" ["003"]
    "that"  ["002", "003"]
    "the"   ["001", "004"]

_Nota: usa la linea:_

    key, text = json.loads(line)

_para obtener extraer el id del documento y el texto del mismo del formato **json** _

In [6]:
# Escribimos el código en un archivo ejecutable
%%writefile InvertedIndex.py

from mrjob.job import MRJob
import json # Importamos la librería json para trabajar con las lineas de entrada en formato JSON

class InvertedIndex(MRJob): # Definimos la clase para "heredar" las funcionalidades de MRJob

    def mapper(self, _, line):
        key, text = json.loads(line)  # Extraemos el id del documento y el texto del formato JSON
        palabras = text.split()  # Separamos el texto en palabras usando line.split() para dividir
                                 # la línea en palabras usando espacios

        for palabra in palabras: # Iteramos sobre cada palabra en el conjunto de palabras del texto
            yield palabra.lower(), key  # Retornamos la palabra que sería la clave convertida a minúsculas
                                        # gracias a la función .lower() para unificar las palabras y el
                                        # valor que sería el ID del documento donde se encontó la palabra

    def reducer(self, key, value): # En este caso la clave es la palabra y el valor son los ID's
        ID_unicos = sorted(set(value))  # Obtenemos y ordenamos los ID únicos con la función
                                       # set() para eliminar duplicados y sorted() para ordenarlos
        yield key, ID_unicos  # Retornamos la palabra (clave) y la lista de los ID únicos (valor)

if __name__ == '__main__':
    InvertedIndex.run()

Writing InvertedIndex.py


Verifica que la salida es la esperada y prueba con nuevos valores.


In [7]:
%%script python InvertedIndex.py -q
["001", "the car is nice"]
["002", "that car is mine"]
["003", "that shirt is nice"]
["004", "the car is the best"]

"mine"	["002"]
"nice"	["001", "003"]
"shirt"	["003"]
"that"	["002", "003"]
"the"	["001", "004"]
"best"	["004"]
"car"	["001", "002", "004"]
"is"	["001", "002", "003", "004"]


In [8]:
%%script python InvertedIndex.py -q
["001", "War is a state of armed conflict between autonomous organizations (such as states and non-state actors) or coalitions of such organizations. It is generally characterized by extreme collective aggression, destruction, and usually high mortality. The set of techniques used by a group to carry out war is known as warfare. An absence of war is usually called peace."]
["032", "War must entail some degree of confrontation using weapons and other military technology and equipment by armed forces employing military tactics and operational art within a broad military strategy subject to military logistics."]
["105", "While some scholars see warfare as a universal and ancestral aspect of human nature, others argue that it is only a result of specific socio-cultural or ecological circumstances"]

"circumstances"	["105"]
"coalitions"	["001"]
"collective"	["001"]
"conflict"	["001"]
"confrontation"	["032"]
"degree"	["032"]
"destruction,"	["001"]
"ecological"	["105"]
"employing"	["032"]
"entail"	["032"]
"equipment"	["032"]
"extreme"	["001"]
"forces"	["032"]
"generally"	["001"]
"group"	["001"]
"high"	["001"]
"human"	["105"]
"is"	["001", "105"]
"it"	["001", "105"]
"known"	["001"]
"logistics."	["032"]
"military"	["032"]
"mortality."	["001"]
"must"	["032"]
"nature,"	["105"]
"non-state"	["001"]
"of"	["001", "032", "105"]
"only"	["105"]
"operational"	["032"]
"or"	["001", "105"]
"organizations"	["001"]
"organizations."	["001"]
"other"	["032"]
"others"	["105"]
"out"	["001"]
"peace."	["001"]
"result"	["105"]
"scholars"	["105"]
"see"	["105"]
"set"	["001"]
"socio-cultural"	["105"]
"some"	["032", "105"]
"specific"	["105"]
"state"	["001"]
"states"	["001"]
"strategy"	["032"]
"subject"	["032"]
"such"	["001"]
"tactics"	["032"]
"techniques"	["001"]
"technology"	["032"]
"that"	["105"]
"the"	["001"]


**Ejemplos de aplicaciones:**
* Puede usarse en motores de búsqueda web indexando páginas para permitir búsquedas rápidas y eficientes.
* Aplicaciones en sistemas de gestión de docuemntos empresariales almacenados en sistemas de archivos, facilitando la búsqueda.
* Creando índices de productos basados en descripciones en comercios electrónicos.
* Indexando videos o contenido educativo en plataformas de aprendizaje en línea para hacer recomendaciones a los estudiantes.

### Ejercicio 4: Suma de Matrices

Representamos dos matrices **a** y **b** de 2x2 de la siguiente forma:

    ["a", 0, 0, 32]
    ["a", 0, 1, 69]
    ["a", 1, 0, 18]
    ["a", 1, 1, 28]
    ["b", 0, 0, 18]
    ["b", 0, 1, 69]
    ["b", 1, 0, 28]
    ["b", 1, 1, 32]

es decir

    a = [32 69]     b = [18 69]    a + b = [50 138]
        [18 28]         [28 32]            [46  60]
        
        
tienes que implementar un programa map-reduce que acepte como entrada las dos matrices y realice la suma

In [9]:
# Escribimos el código en un archivo ejecutable
%%writefile MatrixSum.py

from mrjob.job import MRJob
import json # Importamos la librería json para trabajar con las lineas de entrada en formato JSON

class MatrixSum(MRJob): # Definimos la clase para "heredar" las funcionalidades de MRJob

    def mapper(self, _, line):
        matriz, x, y, val = json.loads(line)  # Extraemos los valores de la matriz del archivo JSON
                                              # en este caso line será un cadena  de JSON que representa
                                              # una matriz con las coordenadas (x, y) y su valor

        yield (x, y), val  # Retornamos la posición (x, y) como clave y el valor de la matriz relacionado

    def reducer(self, key, values):
        suma = sum(values)  # Sumamos los valores de las dos matrices en la misma posición o clave
        yield key, suma  # Retornamos la clave (posición) y la suma de los valores como resultado

if __name__ == '__main__':
    MatrixSum.run()

Writing MatrixSum.py


In [10]:
%%script python MatrixSum.py -q
["a", 0, 0, 32]
["a", 0, 1, 69]
["a", 1, 0, 18]
["a", 1, 1, 28]
["b", 0, 0, 18]
["b", 0, 1, 69]
["b", 1, 0, 28]
["b", 1, 1, 32]

[0, 1]	138
[1, 0]	46
[1, 1]	60
[0, 0]	50


**Ejemplos de aplicaciones:**
* Análisis de datos geoespaciales y aplicación a los Sistemas de Información Geográfica (GIS) con datos de sensores remotos
* Procesamiento de imágenes médicas, para imágenes de resonancia magnética
* Análisis de redes sociales para diferentes tipos de interacciones
* Análisis financiero, combinando matrices de correlación de diversos períodos
* Procesamiento de lenguaje natural
* Análisis de tráfico y transporte, uniendo datos de sistemas de gestión de tráfico con información GPS